In [1]:
!pip install pandas==0.23.4

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
%matplotlib inline

import psycopg2
from itertools import chain
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re
import pytz
import math as math



import sys
from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role

In [4]:
sagemaker_session = sagemaker.Session()

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [6]:
s3_bucket = 'mtgmlbucket'  # replace with an existing bucket if needed
s3_prefix = 'mtgml-notebook'    # prefix used for all data stored within the bucket

role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

In [7]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)


In [8]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [10]:
cur=conn.cursor()

In [11]:
analyname='Supreme Verdict'

cur.execute('''SELECT carddate,price,cardtype FROM public.totalgoatbot WHERE 
cardname IN(SELECT DISTINCT(cardname) FROM public.tourninfo) AND cardname ILIKE %s''',(analyname,))

In [12]:
goatbotrows=cur.fetchall()

In [13]:
goatdf=pd.DataFrame(goatbotrows,columns=["carddate","price","cardtype"]).set_index("carddate")

In [14]:
print(goatdf)

            price  cardtype
carddate                   
2020-01-03  5.20   0       
2020-05-02  4.84   0       
2020-05-01  4.82   0       
2020-04-30  4.87   0       
2020-04-29  4.73   0       
2020-04-28  4.64   0       
2020-04-27  4.70   0       
2020-04-26  4.61   0       
2020-04-25  4.36   0       
2020-04-24  4.02   0       
2020-04-23  3.90   0       
2020-04-22  3.88   0       
2020-04-21  3.67   0       
2020-04-20  3.83   0       
2020-04-19  3.83   0       
2020-04-18  3.92   0       
2020-04-17  3.80   0       
2020-04-16  3.75   0       
2020-04-15  3.60   0       
2020-04-14  3.66   0       
2020-04-13  3.68   0       
2020-04-12  3.85   0       
2020-04-11  4.53   0       
2020-04-10  4.91   0       
2020-04-09  4.77   0       
2020-04-08  4.85   0       
2020-04-07  5.49   0       
2020-04-06  4.79   0       
2020-04-05  4.29   0       
2020-04-04  4.53   0       
2020-04-03  4.52   0       
2020-04-02  4.42   0       
2020-04-01  4.63   0       
2020-03-31  4.70   0

In [15]:
goatdfarr=[]
for ctype in sorted(goatdf.cardtype.unique()):
    goatdfarr.append(goatdf[goatdf['cardtype']==ctype])
    goatdfarr[ctype]=goatdfarr[ctype].drop(columns='cardtype')
    goatdfarr[ctype].columns=['price'+str(ctype)]

In [16]:
goatdf

,price,cardtype
carddate,,
2020-01-03,5.20,0
2020-05-02,4.84,0
2020-05-01,4.82,0
2020-04-30,4.87,0
2020-04-29,4.73,0
2020-04-28,4.64,0
2020-04-27,4.70,0
2020-04-26,4.61,0
2020-04-25,4.36,0


In [17]:
cur.execute('''SELECT carddate,AVG(pshipq) as AVGE,percentile_cont(0.5) WITHIN GROUP (ORDER BY pshipq) as MED
    FROM (SELECT DISTINCT title,cardname,carddate,(price+shipping)/COALESCE(cardquantity,maybecardquantity,1) as pshipq FROM public.transactions
	WHERE possiblybad IS FALSE 
    AND cardname ILIKE %s
	AND isfoil IS FALSE
	AND (cardlanguage='english' or cardlanguage IS NULL)
	AND cardset NOT IN('LEA','LEB','U')
	AND (isemblem IS NULL OR isemblem IS FALSE)
	AND (lotis IS FALSE OR lotis IS NULL)
	AND (isboosterbox IS NULL OR isboosterbox IS FALSE)
	AND cardspecial IS NULL
	AND issleeve IS NULL
	AND isdeck IS NULL
	AND isplaymat IS NULL
	AND isultra IS NULL
	AND (istoken IS NULL OR istoken IS FALSE)
	AND (iscommanderdeck IS NULL OR iscommanderdeck IS FALSE)
	AND isgraded IS NULL
	AND (othercards IS NULL OR transplit IS TRUE)
	AND saletype IN('normal')
	AND price+shipping<19) as foo GROUP BY carddate ORDER BY carddate ASC''',(analyname,))

In [18]:
ebayrows=cur.fetchall()

In [19]:
#ebaydf=pd.DataFrame(ebayrows,columns=["carddate","avg","med"]) 
ebaydf=pd.DataFrame(ebayrows,columns=["carddate","AVG","MED"]).set_index("carddate")
ebaydf['MINAVGMED']=ebaydf.apply(lambda row: np.minimum(row.AVG,row.MED), axis = 1)
ebaydf=ebaydf.drop(columns=['AVG','MED'])

In [20]:
print(ebaydf)

            MINAVGMED
carddate             
2019-10-21  5.245000 
2019-10-22  6.000000 
2019-10-23  5.290000 
2019-10-24  6.295000 
2019-10-25  6.118000 
2019-10-26  8.940001 
2019-10-27  7.243333 
2019-10-28  6.833333 
2019-10-30  6.880000 
2019-10-31  5.485000 
2019-11-01  7.175000 
2019-11-02  7.620000 
2019-11-03  7.820000 
2019-11-04  7.475000 
2019-11-05  8.495000 
2019-11-06  9.520000 
2019-11-07  7.923333 
2019-11-08  7.755000 
2019-11-11  10.680000
2019-11-12  9.186250 
2019-11-13  10.340000
2019-11-14  10.790000
2019-11-15  9.811666 
2019-11-16  10.059999
2019-11-17  10.495000
2019-11-18  11.377778
2019-11-19  11.677500
2019-11-21  11.790000
2019-11-22  10.300000
2019-11-23  10.000000
2019-11-24  4.330000 
2019-11-25  11.763333
2019-11-26  9.990000 
2019-11-28  11.975000
2019-11-29  11.000000
2019-11-30  11.506000
2019-12-01  14.480000
2019-12-02  12.740000
2019-12-03  11.000000
2019-12-04  11.790000
2019-12-05  11.490000
2019-12-07  11.590000
2019-12-09  11.576667
2019-12-10

In [21]:
cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
(SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.tourninfo WHERE cardname ILIKE %s) AS foo1,
(SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.tourninfo GROUP BY entrydate) AS bar
WHERE entrydate0=entrydate1) AS foo
GROUP BY cardname0,entrydate1''',(analyname,))

In [22]:
tournrows=cur.fetchall()

In [23]:
tourndf=pd.DataFrame(tournrows,columns=["carddate","ratio"]).set_index("carddate")
tourndf=tourndf.astype({'ratio': 'float64'})

In [24]:
print(tourndf)

               ratio
carddate            
2019-10-27  0.003750
2019-10-28  0.006046
2019-10-31  0.008246
2019-11-01  0.001667
2019-11-03  0.005000
2019-11-04  0.005972
2019-11-07  0.005557
2019-11-09  0.005838
2019-11-10  0.009163
2019-11-11  0.005086
2019-11-14  0.006987
2019-11-15  0.011250
2019-11-17  0.003769
2019-11-18  0.005506
2019-11-21  0.005520
2019-11-24  0.007222
2019-11-25  0.007543
2019-11-26  0.007923
2019-11-28  0.005228
2019-11-29  0.007503
2019-11-30  0.003333
2019-12-01  0.002271
2019-12-02  0.003489
2019-12-05  0.006306
2019-12-07  0.005000
2019-12-08  0.010014
2019-12-10  0.004680
2019-12-12  0.005186
2019-12-14  0.002667
2019-12-15  0.011314
2019-12-16  0.007181
2019-12-17  0.019210
2019-12-18  0.011429
2019-12-19  0.004089
2019-12-20  0.001905
2019-12-21  0.011439
2019-12-22  0.010797
2019-12-23  0.004660
2019-12-24  0.003955
2019-12-25  0.005333
2019-12-26  0.002945
2019-12-27  0.001250
2019-12-28  0.005330
2019-12-29  0.004167
2019-12-30  0.002291
2019-12-31  0

In [25]:

cur.close()
conn.close()

In [26]:
freq='D'
prediction_length=1
context_length=7

start_date = pd.Timestamp("2019-10-23", freq=freq)
end_training = pd.Timestamp("2020-09-29", freq=freq)
end_dataset = pd.Timestamp("2020-09-29", freq=freq)
mid_dataset=pd.Timestamp("2020-09-15",freq=freq)
end_testing = end_training + prediction_length*end_training.freq
print(f'End training: {end_training}, End testing: {end_testing}')

End training: 2020-09-29 00:00:00, End testing: 2020-09-30 00:00:00


In [27]:
goatdfarr.insert(0,ebaydf)
goatdfarr.append(tourndf)
overalldf=pd.concat(goatdfarr,axis=1,sort=True)

In [28]:
overalldf.index=pd.to_datetime(overalldf.index,infer_datetime_format=True)

In [29]:
overalldf.index.name='carddate'
overalldf=pd.concat([overalldf['MINAVGMED'].fillna("NaN"),
                     overalldf.loc[:, overalldf.columns.difference(['MINAVGMED'])].fillna(method='ffill').fillna(method='bfill')],
                    axis=1,join='inner')

In [30]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))


In [31]:
dynamic_feat_arr=[]
for col in overalldf.columns[1:]:
    dynamic_feat_arr.append(list(overalldf[start_date-timedelta(1):end_training-timedelta(1)][col]))

In [32]:
training_data=[{"start":str(start_date),
               "target":list(overalldf[start_date:end_training]['MINAVGMED']),
               "dynamic_feat":dynamic_feat_arr}]


In [33]:
write_dicts_to_file("train.json", training_data)
# write_dicts_to_s3(f'{s3_data_path}/test/test.json',)

In [34]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)


In [35]:
copy_to_s3("train.json", s3_data_path + "/train/train.json")
#copy_to_s3("test.json", s3_data_path + "/test/test.json")

Uploading file to s3://mtgmlbucket/mtgml-notebook/data/train/train.json


In [36]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

{"start": "2019-10-23 00:00:00", "target": [5.28999996185303, 6.29499983787537, 6.11800007820129, 8....


In [37]:
overalldf

,MINAVGMED,price0,price1,price2,price3,price4,price5,ratio
carddate,,,,,,,,
2017-06-29,NaN,2.14,1.49,3.05,2.53,1.31,10.99,0.003750
2017-06-30,NaN,2.50,1.49,3.10,2.53,1.31,10.99,0.003750
2017-07-01,NaN,2.85,1.49,3.26,2.28,1.31,10.99,0.003750
2017-07-02,NaN,2.86,1.49,3.26,1.83,1.31,10.99,0.003750
2017-07-03,NaN,3.06,1.49,3.28,2.05,1.31,10.99,0.003750
2017-07-04,NaN,3.57,1.49,3.41,2.41,1.31,10.99,0.003750
2017-07-05,NaN,3.54,1.49,3.70,2.57,1.31,10.99,0.003750
2017-07-06,NaN,3.79,1.49,3.77,2.57,1.31,10.99,0.003750
2017-07-07,NaN,3.84,1.49,3.61,2.57,1.31,10.99,0.003750


In [38]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    base_job_name='deepar-mtgml',
    output_path=s3_output_path
)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [39]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length)
}

In [40]:
estimator.set_hyperparameters(**hyperparameters)

In [41]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    #"test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels, wait=True)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-03 17:46:25 Starting - Starting the training job...
2020-10-03 17:46:27 Starting - Launching requested ML instances......
2020-10-03 17:47:28 Starting - Preparing the instances for training...
2020-10-03 17:48:06 Downloading - Downloading input data...
2020-10-03 17:48:54 Training - Training image download completed. Training in progress..Arguments: train
[10/03/2020 17:48:55 INFO 139787867731776] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u'

In [42]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + timedelta(1)
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

In [43]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

In [44]:
dynamic_feat_pred=[]
for col in overalldf.columns[1:]:
    dynamic_feat_pred.append(list(overalldf[start_date-timedelta(1):end_dataset][col]))

In [85]:
predictor.predict(ts=overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').resample('D').mean(),dynamic_feat=dynamic_feat_pred, quantiles=[0.10, 0.5, 0.90])["0.5"]

2020-09-30    4.781867
Freq: D, Name: 0.5, dtype: float64

In [81]:
def plot(
    predictor, 
    target_ts, 
    cat=None, 
    dynamic_feat=None, 
    forecast_date=end_training, 
    show_samples=False, 
    plot_history=7,
    confidence=80
):
    print("calling served model to generate predictions starting from {}".format(str(forecast_date)))
    assert(confidence > 50 and confidence < 100)
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
        
    # we first construct the argument to call our model
    args = {
        "ts": target_ts[:forecast_date],
        "return_samples": show_samples,
        "dynamic_feat":dynamic_feat,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100
    }


    if dynamic_feat is not None:
        args["dynamic_feat"] = dynamic_feat
        fig = plt.figure(figsize=(20, 6))
        ax = plt.subplot(2, 1, 1)
    else:
        fig = plt.figure(figsize=(20, 3))
        ax = plt.subplot(1,1,1)
    
    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, 'cat = {}'.format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples: 
        for key in prediction.keys():
            if "sample" in key:
                #prediction[key].plot(color='lightskyblue', alpha=0.2, label='_nolegend_')
                plt.plot(prediction[key])
                
                
    # plot the target
    target_section = target_ts[forecast_date-plot_history:forecast_date+prediction_length]
    plt.plot(target_section)

    
    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label='{}% confidence interval'.format(confidence)
    )
    print(prediction["0.5"])
    #prediction["0.5"].plot(color="b", label='P50').scatter()
    plt.plot(prediction["0.5"])
    ax.legend(loc=2)    
    
    # fix the scale as the samples may change it
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)
    
    if dynamic_feat is not None:
        for i, f in enumerate(dynamic_feat, start=1):
            ax = plt.subplot(len(dynamic_feat) * 2, 1, len(dynamic_feat) + i, sharex=ax)
            feat_ts = pd.Series(
                index=pd.DatetimeIndex(start=target_ts.index[0], freq=target_ts.index.freq, periods=len(f)),
                data=f
            )
            #feat_ts[forecast_date-plot_history:forecast_date+prediction_length].plot(ax=ax, color='g')
            plt.plot(feat_ts[forecast_date-plot_history:forecast_date+prediction_length])



In [82]:
style = {'description_width': 'initial'}

In [83]:
@interact_manual(
    forecast_day=IntSlider(min=0, max=365, value=1, style=style),
    confidence=IntSlider(min=60, max=95, value=80, step=5, style=style),
    history_weeks_plot=IntSlider(min=0, max=10, value=1, style=style),
    show_samples=Checkbox(value=False),
    continuous_update=False
)
def plot_interact( forecast_day, confidence, history_weeks_plot, show_samples):
    plot(
        predictor,
        dynamic_feat=dynamic_feat_pred,
        target_ts=overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').resample('D').mean(),
        forecast_date=end_training + datetime.timedelta(days=forecast_day),
        show_samples=show_samples,
        plot_history=history_weeks_plot*7,
        confidence=confidence
    )


interactive(children=(IntSlider(value=1, description='forecast_day', max=365, style=SliderStyle(description_wi…

In [67]:
overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').resample('D').mean()

carddate
2019-10-23    5.290000 
2019-10-24    6.295000 
2019-10-25    6.118000 
2019-10-26    8.940001 
2019-10-27    7.243333 
2019-10-28    6.833333 
2019-10-29   NaN       
2019-10-30    6.880000 
2019-10-31    5.485000 
2019-11-01    7.175000 
2019-11-02    7.620000 
2019-11-03    7.820000 
2019-11-04    7.475000 
2019-11-05    8.495000 
2019-11-06    9.520000 
2019-11-07    7.923333 
2019-11-08    7.755000 
2019-11-09   NaN       
2019-11-10   NaN       
2019-11-11    10.680000
2019-11-12    9.186250 
2019-11-13    10.340000
2019-11-14    10.790000
2019-11-15    9.811666 
2019-11-16    10.059999
2019-11-17    10.495000
2019-11-18    11.377778
2019-11-19    11.677500
2019-11-20   NaN       
2019-11-21    11.790000
2019-11-22    10.300000
2019-11-23    10.000000
2019-11-24    4.330000 
2019-11-25    11.763333
2019-11-26    9.990000 
2019-11-27   NaN       
2019-11-28    11.975000
2019-11-29    11.000000
2019-11-30    11.506000
2019-12-01    14.480000
2019-12-02    12.740000
2019-12